# Clustering and Similarity

In [ ]:
%matplotlib inline 

import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

sns.set_context('notebook')
sns.set_style('whitegrid')

## Distance and Similarity

### Minkowski Distance 

### Mahalanobis Distance 

### Cosine Distance 

### Other Distance Metrics

## Clustering Metrics

### Silhouette Scores

### Inter-Cluster Distance Maps

### Rand Index

## Agglomerative Clustering

## Partitive Clustering

## Curse of Dimensionality

### No Free Lunch